In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore') 
pd.set_option('display.max_columns',None)

In [2]:
df = pd.read_csv('Flight_Fare.csv')
df.head(3)
                 

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


Basic check

In [3]:
df.shape

(10683, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [6]:
df.describe(include = ['O'])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
count,10683,10683,10683,10683,10682,10683,10683,10683,10682,10683
unique,12,44,5,6,128,222,1343,368,5,10
top,Jet Airways,18/05/2019,Delhi,Cochin,DEL ? BOM ? COK,18:55,19:00,2h 50m,1 stop,No info
freq,3849,504,4537,4537,2376,233,423,550,5625,8345


EDA

In [69]:
import sweetviz as sv
my_rep = sv.analyze(df)
my_rep.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Preprocessing


In [7]:
df.duplicated().sum()

220

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.duplicated().sum()

0

In [10]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [11]:
df.Route.value_counts()

DEL ? BOM ? COK          2376
BLR ? DEL                1536
CCU ? BOM ? BLR           979
CCU ? BLR                 724
BOM ? HYD                 621
                         ... 
CCU ? VTZ ? BLR             1
CCU ? IXZ ? MAA ? BLR       1
BOM ? COK ? MAA ? HYD       1
BOM ? CCU ? HYD             1
BOM ? BBI ? HYD             1
Name: Route, Length: 128, dtype: int64

In [12]:
df.loc[df['Route'].isnull()==True,'Route'] = 'DEL ? BOM ? COK'

In [13]:
df.Total_Stops.value_counts()

1 stop      5625
non-stop    3475
2 stops     1318
3 stops       43
4 stops        1
Name: Total_Stops, dtype: int64

In [14]:
df.loc[df['Total_Stops'].isnull() == True,'Total_Stops']='1 stop'

In [15]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [16]:
#encoding
df['journey_day'] = pd.to_datetime(df.Date_of_Journey,format = '%d/%m/%Y').dt.day

In [17]:
df['journey_month'] = pd.to_datetime(df.Date_of_Journey,format = '%d/%m/%Y').dt.month

In [18]:
df.drop('Date_of_Journey',axis = 1,inplace =  True)
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month
0,IndiGo,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5


In [19]:
df['Dep_hour'] = pd.to_datetime(df.Dep_Time).dt.hour

In [20]:
df['Dep_min'] = pd.to_datetime(df.Dep_Time).dt.minute

In [21]:
df.drop('Dep_Time',axis = 1,inplace = True)
df.head(3)

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR ? DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,22,20
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,13:15,7h 25m,2 stops,No info,7662,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,04:25 10 Jun,19h,2 stops,No info,13882,9,6,9,25


In [22]:
df['Arrival_hour'] = pd.to_datetime(df.Arrival_Time).dt.hour

In [23]:
df['Arrival_min'] = pd.to_datetime(df.Arrival_Time).dt.minute

In [24]:
df.drop('Arrival_Time',axis = 1,inplace = True)
df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR ? DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,15


In [25]:
df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR ? DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,15


In [26]:
duration = list(df["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"
        else:
            duration[i] = "0h " + duration[i]

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))



df["Duration_hours"] = duration_hours
df["Duration_mins"] = duration_mins
df.drop(["Duration"], axis = 1, inplace = True)

In [27]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,journey_day,journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,BLR ? DEL,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2 stops,No info,7662,1,5,5,50,13,15,7,25


In [28]:
#encoding
df.Airline.value_counts()

Jet Airways                          3700
IndiGo                               2043
Air India                            1695
Multiple carriers                    1196
SpiceJet                              815
Vistara                               478
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [29]:
Airline = pd.get_dummies(df.Airline,drop_first = True)

In [30]:
Airline.head(3)

,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Trujet,Vistara,Vistara Premium economy
0,0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0


In [31]:
df.Source.value_counts()

Delhi       4346
Kolkata     2860
Banglore    2179
Mumbai       697
Chennai      381
Name: Source, dtype: int64

In [32]:
Source = pd.get_dummies(df.Source,drop_first = True)

In [33]:
Source.head(3)


,Chennai,Delhi,Kolkata,Mumbai
0,0,0,0,0
1,0,0,1,0
2,0,1,0,0


In [34]:
cole = ({'Chennai':'Che','Delhi':'Del','Kolkata':'Kol','Mumbai':'Mum'})
Source.rename(columns = cole,inplace = True)

In [35]:
df.Destination.value_counts()

Cochin       4346
Banglore     2860
Delhi        1265
New Delhi     914
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

In [36]:
Destination = pd.get_dummies(df.Destination,drop_first = True)

In [37]:
Destination.head(2)

,Cochin,Delhi,Hyderabad,Kolkata,New Delhi
0,0,0,0,0,1
1,0,0,0,0,0


In [38]:
df.Total_Stops.value_counts()

1 stop      5626
non-stop    3475
2 stops     1318
3 stops       43
4 stops        1
Name: Total_Stops, dtype: int64

In [39]:
df.replace({'1 stop':1,'non-stop':0,'2 stops':2,'3 stops':3,'4 stops':4},inplace= True)


In [40]:
df1 = pd.concat([df,Airline, Source, Destination],axis = 1)

In [41]:
df1.head(2)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,journey_day,journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Trujet,Vistara,Vistara Premium economy,Che,Del,Kol,Mum,Cochin,Delhi,Hyderabad,Kolkata,New Delhi
0,IndiGo,Banglore,New Delhi,BLR ? DEL,0,No info,3897,24,3,22,20,1,10,2,50,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2,No info,7662,1,5,5,50,13,15,7,25,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [42]:
drop = ['Airline','Source','Destination','Route','Additional_Info']
df1.drop(drop,axis = 1 ,inplace = True)

In [43]:
df1.head(2)

,Total_Stops,Price,journey_day,journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Trujet,Vistara,Vistara Premium economy,Che,Del,Kol,Mum,Cochin,Delhi,Hyderabad,Kolkata,New Delhi
0,0,3897,24,3,22,20,1,10,2,50,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,7662,1,5,5,50,13,15,7,25,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [44]:
sel = ['Air India','GoAir','IndiGo','Jet Airways','Jet Airways Business',
          'Multiple carriers','Multiple carriers Premium economy','SpiceJet',
          'Trujet','Vistara','Vistara Premium economy','journey_day',
          'journey_month','Che','Del','Kol','Mum','Cochin',
          'Delhi','Hyderabad','Kolkata','New Delhi','Dep_hour','Dep_min',
          'Arrival_hour','Arrival_min','Duration_hours','Duration_mins','Total_Stops','Price']

df1 = df1[sel]

In [45]:
df1.head(2)

,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,Trujet,Vistara,Vistara Premium economy,journey_day,journey_month,Che,Del,Kol,Mum,Cochin,Delhi,Hyderabad,Kolkata,New Delhi,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins,Total_Stops,Price
0,0,0,1,0,0,0,0,0,0,0,0,24,3,0,0,0,0,0,0,0,0,1,22,20,1,10,2,50,0,3897
1,1,0,0,0,0,0,0,0,0,0,0,1,5,0,0,1,0,0,0,0,0,0,5,50,13,15,7,25,2,7662


In [46]:
df1.duplicated().sum()


2

In [47]:
df1.drop_duplicates(inplace = True)

In [48]:
x = df1.drop('Price',axis = 1)
y = df1['Price']

In [49]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.30,random_state = 42)

In [50]:
x_train.shape

(7322, 29)

In [51]:
y_train.shape

(7322,)

In [52]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators = 500)
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [53]:
y_pred = model.predict(x_test)
y_pred

array([14937.767 ,  3944.6187,  3902.7905, ..., 13461.393 ,  4842.285 ,
       15663.586 ], dtype=float32)

In [54]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.7927650722485694

In [55]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [56]:
MSE = mean_squared_error(y_test,y_pred)
MSE

4358331.68445189

In [57]:
MAE = mean_absolute_error(y_test,y_pred)
MAE

1294.2653474932304

In [59]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(100, 2000, 11)]
max_depth = [1, 2, 3, 4, 5]
learning_rate = [0.1, 0.2, 0.3, 0.5]
gamma = [1, 2, 3, 4, 5, 6]



params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate,
          'gamma': gamma}


model1 = XGBRegressor(random_state=42)


cv = RandomizedSearchCV(model1, params, scoring='f1', n_jobs=-1, cv=7, n_iter=30, random_state=42)


In [60]:
cv.fit(x_train,y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=42, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'gamma': [1, 2, 3, 4, 5, 6],
                                        'learning_rate': [0.1, 0.2, 0.3, 0.5],
                                        'max_depth': [1, 2, 3, 4, 5],
                                        'n_estimators': [100, 290, 480, 670,
                                                         860, 1050, 1240, 1430,
                                                         1620, 1810, 2000]},
                   random_state=42, scoring='f1')

In [63]:
best_para = cv.best_params_
print(f'Best paras:{best_para}')

Best paras:{'n_estimators': 1430, 'max_depth': 2, 'learning_rate': 0.1, 'gamma': 4}


In [64]:
model3 = XGBRegressor(n_estimators =  1430, max_depth = 2, learning_rate = 0.1, gamma =  4)
model3.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=4, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1430, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [66]:
y_p = model3.predict(x_test)
y_p

array([14227.122 ,  3630.7605,  5078.132 , ..., 19766.154 ,  4339.9937,
       13284.8955], dtype=float32)

In [68]:
r2_score(y_test,y_p)

0.7958629469820526